# 📘 Feature Store Registration
This notebook registers the engineered sensor features table from the Delta Live Tables (DLT) pipeline
into the Databricks Feature Store for easier discoverability, governance, and reuse during model training and inference.


In [0]:
df = spark.table("arao.aerodemo.sensor_features")
df.write.format("delta").mode("overwrite").saveAsTable("arao.aerodemo.sensor_features_table")

In [0]:
%sql
ALTER TABLE arao.aerodemo.sensor_features_table 
ALTER COLUMN aircraft_id SET NOT NULL;

ALTER TABLE arao.aerodemo.sensor_features_table 
ALTER COLUMN timestamp SET NOT NULL;

In [0]:
%sql
ALTER TABLE arao.aerodemo.sensor_features_table 
ADD CONSTRAINT sensor_features_pk 
PRIMARY KEY (aircraft_id, timestamp);

In [0]:
from databricks.feature_store import FeatureStoreClient

fs = FeatureStoreClient()

# Load the Delta table as a DataFrame
df = spark.read.table("arao.aerodemo.sensor_features_table")

# Register the table into Unity Catalog Feature Store
fs.create_table(
    name="arao.aerodemo.sensor_features_table",  # Fully qualified UC path
    primary_keys=["aircraft_id", "timestamp"],
    timestamp_keys=["timestamp"],
    df=df,
    description="ML features engineered from aircraft sensor and maintenance data"
)